In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer



In [2]:
data_path = "Dataset_analyse.csv"
df = pd.read_csv(data_path)

df = df.dropna()
df = df.drop(["title","total_spectator","hebdo_rank","first_weekend","rating_public","casting","visa","award","lang","classification_acteurs"], axis=1)
df

,year,director,country,duration,genre,first_day,first_week,copies,distributor,rating_press,budget
0,1997,3,france,6300,thriller,180000.6,172230,234,2,3.26,89800000.0
1,1999,3,france,5400,drame,34301.6,154881,198,3,3.40,990000000.0
2,1994,3,france,5220,comedie,190299.6,417021,189,3,2.74,113600000.0
3,1995,3,etatsunis,9900,aventure action,39170.4,355642,302,3,2.60,72000000.0
4,1994,3,france,6600,thriller,71377.6,39739,46,3,3.48,687840000.0
...,...,...,...,...,...,...,...,...,...,...,...
2202,2008,3,grandebretagne,7080,comedie,18126.0,110442,128,3,3.90,44600000.0
2203,1993,1,etatsunis,6060,drame,174755.0,32351,44,3,3.20,46000000.0
2204,1999,3,france,5520,drame,156540.2,51749,-1,3,3.90,182000000.0
2205,1999,3,etatsunis,7080,comedie,125198.8,63249,-1,3,3.80,86262750.8


In [3]:
X = df.drop(['first_week'], axis=1)
y = df.first_week

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42)

num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore', sparse_output=False), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)


In [4]:
from sklearn import set_config


set_config(transform_output="pandas")
model = make_pipeline(
    my_final_pipeline,
)
xdata = model.fit_transform(X_train, y_train)
xdata.head()

,1,categorical__country_allemagne,categorical__country_australie,categorical__country_belgique,categorical__country_bresil,categorical__country_canada,categorical__country_chine,categorical__country_danemark,categorical__country_espagne,categorical__country_etatsunis,...,numerical__copies^2,numerical__copies numerical__distributor,numerical__copies numerical__rating_press,numerical__copies numerical__budget,numerical__distributor^2,numerical__distributor numerical__rating_press,numerical__distributor numerical__budget,numerical__rating_press^2,numerical__rating_press numerical__budget,numerical__budget^2
1896,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,6.763612,7.802083,10.691744,1.717764,9.0,12.333333,1.981506,16.901235,2.715397,0.436263
1077,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.929012,4.166667,4.938272,2.816307,9.0,10.666667,6.083223,12.641975,7.209746,4.111734
454,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.176517,1.260417,1.493827,0.111001,9.0,10.666667,0.792602,12.641975,0.939381,0.069802
652,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.399354,1.895833,1.909877,0.351451,9.0,9.066667,1.668428,9.133827,1.680787,0.309295
465,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.882921,1.697917,4.716435,0.437376,1.0,2.777778,0.257596,7.716049,0.715544,0.066356


In [5]:

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

# param_grid = {'lasso__alpha': [15]}


# model = GridSearchCV(model, param_grid,cv =5)
model.fit(X_train, y_train)
grid_score = model.score(X_train, y_train)

# best_alpha = model.best_params_['lasso__alpha']

/home/utilisateur/Documents/Projets/Brief_Popularity_Film/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.730e+13, tolerance: 3.753e+10
  model = cd_fast.enet_coordinate_descent(


In [6]:
# y_pred_train = model.predict(X_train)


# residus =  y_pred_train - y_train


# weights = np.exp(-abs(residus) / residus.std())
# model.fit(X_train, y_train, lasso__sample_weight=weights)




y_pred = model.predict(X_test)

In [7]:

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle :")
# print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle :
R2 Score: 0.8599
MAE: 127739.94
RMSE: 222230.58


In [8]:
# -------------------------------------
# ---------------------------------------
# -----------------------------------------
# ------------------------------------------
# -------------------------------------------
# -------------------------------------
# ---------------------------------------
# -----------------------------------------
# ------------------------------------------
# -------------------------------------------
# -------------------------------------
# ---------------------------------------
# -----------------------------------------
# ------------------------------------------
# -------------------------------------------
# -------------------------------------
# ---------------------------------------
# -----------------------------------------
# ------------------------------------------
# -------------------------------------------

In [9]:
dfdf = pd.DataFrame(X_test)
dfdf


,year,director,country,duration,genre,first_day,copies,distributor,rating_press,budget
574,1993,3,etatsunis,5760,horreur,32137.6,72,3,3.22,11000000.0
2082,2006,3,france,7500,thriller,205622.0,434,3,3.30,11700000.0
443,2010,3,etatsunis,6540,thriller,10079.0,66,3,3.10,13000000.0
557,1996,3,etatsunis,8940,thriller,14894.0,145,3,3.60,40000000.0
202,2009,3,france,5640,drame,18558.0,125,3,3.70,61000000.0
...,...,...,...,...,...,...,...,...,...,...
65,2007,3,france,6600,comedie,43300.0,513,3,2.80,8700000.0
2176,1997,3,france,7200,comedie,91460.8,158,3,3.28,62840000.0
982,2015,3,etatsunis,8160,fantasy,619020.0,1093,3,4.00,38400000.0
1256,2017,3,france,6420,drame,30219.0,196,3,4.10,339600000.0


In [10]:
data = {
    'year': 2024,
    'director': 1,
    'country': "etatsunis",
    'duration': 5640,
    'genre' : "animation",
    'copies': 696,
    'budget': 85000000,
    'classification_acteurs': 2
}

data = {k:[v] for k,v in data.items()}

df_test = pd.DataFrame(data)
df_test

,year,director,country,duration,genre,copies,budget,classification_acteurs
0,2024,1,etatsunis,5640,animation,696,85000000,2


In [11]:
y_pred = model.predict(df_test)

ValueError: columns are missing: {'rating_press', 'distributor', 'first_day'}

In [ ]:
r2_cleaned = r2_score(df_test, y_pred)
mse_cleaned = mean_squared_error(df_test, y_pred)
mae = mean_absolute_error(df_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle :")
# print(model.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

ValueError: could not convert string to float: 'etatsunis'